In [1]:
import requests
from bs4 import BeautifulSoup
import xlwings as xw
from datetime import datetime, date
from datetime import timedelta
import datetime



## 取得n日前日期

In [2]:
def get_date(days):
    # 格式化為 月/日 形式 
    return((date.today() - timedelta(days)).strftime("%m/%d"))

## 獲取兩個日期間所有日期

In [3]:
def getEveryDay(begin_date,end_date):
    date_list = []
    begin_date = datetime.datetime.strptime(begin_date, "%m/%d")
    end_date = datetime.datetime.strptime(end_date,"%m/%d")
    while begin_date <= end_date:
        date_str = begin_date.strftime("%m/%d")
        date_list.append(date_str)
        begin_date += datetime.timedelta(days=1)
    return date_list

## 抓取頁面url

In [4]:
def get_page_url(days,current_page = 'https://www.ptt.cc/bbs/MobileComm/index.html'):
    my_headers = {'cookie': 'over18=1;'}
    resp = requests.get(current_page, headers = my_headers)
    c_content = BeautifulSoup(resp.content, 'html.parser')
    # current page date
    c_date = []
    for c in c_content.find_all('div',class_='r-ent'):
        category = c.find(class_='title').text[:5]
        if ('[公告]') not in category:
            if ('[協尋]') not in category:
                c_date.append(c.find('div', class_='date').text.replace(' ','0'))

            
            
            
    all_url = []   

    while set(c_date) & (set(days)):
        all_url.append(current_page)

        #go to previous page 
        previous_page = 'https://www.ptt.cc' + c_content.find_all('a', class_='btn wide')[1]['href']
        p_resp = requests.get(previous_page, headers = my_headers)
        p_content = BeautifulSoup(p_resp.content, 'html.parser')
        p_date = []
        for p in p_content.find_all('div', class_="date"):
            if p not in p_date:
                p_date.append(p.text.replace(' ', '0'))

        # renew current content,date
        c_content = p_content
        c_date = p_date
        current_page = previous_page
    else:
        return all_url
        
        

----------

##  爬取當前頁面資訊

In [5]:
def ppt_web_crawler(homepage_url, get_date):
    my_headers = {'cookie': 'over18=1;'}
    resp = requests.get(homepage_url, headers = my_headers)
    content = BeautifulSoup(resp.content, 'html.parser')
    category = content.find('a', class_='board').text.replace('看板 ','PTT : ')
    articles_content = content.select(".r-ent")
    # upvote, title, author, date, url
    articles = []
    for ar in articles_content:
        article = {}
        if ar.find('a'):
            
            upvote = ar.find(class_='nrec').text
            title = ar.find('a').text
            url = "https://www.ptt.cc" + ar.find('a')['href']      
            author = ar.find(class_='author').text
            date = ar.find(class_='date').text.replace(' ','0')

            article['source'] = category
            article['upvote'] = upvote
            article['title'] = title
            article['author'] = author
            article['date'] = date
            article['url'] = url
            if date in get_date:
                # 排除公告 
                if ('[公告]' not in article['title'][:4]) and ('[MobileComm]' not in article['title'][:12]):   
                    articles.append(article)

        
    return articles

##  獲取當篇文章內容

In [6]:
def ar_content(url):
    my_headers = {'cookie': 'over18=1;'}
    resp = requests.get(url, headers = my_headers)
    bs = BeautifulSoup(resp.content, 'html.parser')
    content = bs.find(id='main-content')
    #cut = '※ 發信站: 批踢踢實業坊(ptt.cc),'
    cut = '\n--\n'
    
    #有文章內容上方的時間欄才抓取
    try:
    
        p_time_main = content.find_all('div')[3].text.split('時間')[1]    
       
     
        # main_content = content.text.split(cut)[0].split(p_time_main)[1]
        # 0423 modify 主文中以，取代換行'\n'及空白' '    
        main_content = content.text.split(cut)[0].split(p_time_main)[1].replace('\n','，').replace(' ','，')
        
        main_time = content.find_all('span',class_='article-meta-value')[-1].text

        push_content = content.find_all(class_='push')
        # p_content_list include [tag, account, push_content, p_time]
        p_content_list = []
        for p in push_content:
            p_content = []
            tag = p.find_all('span')[0].text
            account = p.find_all('span')[1].text
            if p.find_all('span')[2].text != ':':
                push_content = p.find_all('span')[2].text.split(': ')[1]
            else:
                push_content = ''

            p_time = p.find_all('span')[3].text.replace('\n','')

            p_content.append(tag)
            p_content.append(account)
            p_content.append(push_content)
            p_content.append(p_time)

            p_content_list.append(p_content)


        return main_time,main_content, p_content_list
    
    except:
        print('\033[1;35m @@ Incorrect content! \033[0m') #改變文字顏色
        pass

## Save content to the list

In [7]:
def cp_content(articles):  # save current page content to the list
    
    cp_ar = []
    
    for ar in articles:
        #有內容才append
        if ar_content(ar['url']):
            #0426月 -> /, 日->/
            date = ar['date'].replace('月','/').replace('日','/')
            main_time = ar_content(ar['url'])[0]
            #0426 add year
            year = main_time.split(' ')[-1]
            source = ar['source']
            upvote = ar['upvote']
            url = ar['url']
            author = ar['author']
            title = ar['title']

            main_content = ar_content(ar['url'])[1]
            all_re_content = ar_content(ar['url'])[2]



            #主文
            current_main = []
            #0426 year
            current_main.append(year + '/' + date)            
            current_main.append(main_time)            
            current_main.append(source)
            current_main.append(upvote)
            current_main.append(url)
            current_main.append(author)
            current_main.append(title)
            current_main.append(main_content)
            current_main.append('') # re_tag
            current_main.append('') # re_author
            current_main.append('') # re_content
            current_main.append('') # re_time
            current_main.append('主文') # type

            cp_ar.append(current_main)

            for re in all_re_content:  
                current_re = []    
                # 回文
                #0426 year 
                current_re.append(year + '/' + date)
                current_re.append(main_time)
                current_re.append(source)
                current_re.append(upvote)
                current_re.append(url)
                current_re.append(author)
                current_re.append(title)
                # 0423 type為回文不放主文內容
                #current_re.append(main_content)
                current_re.append('')
                current_re.append(re[0]) # tag
                current_re.append(re[1]) # author
                current_re.append(re[2].replace('=','≡')) # content
                current_re.append(re[3]) # time
                current_re.append('回文') # type
                # 0423 同作者連續回文內容合併
                if current_re[9] == cp_ar[-1][9]:
                    cp_ar[-1][10] = cp_ar[-1][10] + '，' + current_re[10]
                else:
                    cp_ar.append(current_re)       

            print(title)
                
    
    return cp_ar

## 寫入excel

In [24]:
mobile = 'https://www.ptt.cc/bbs/MobileComm/index.html'
gossiping = 'https://www.ptt.cc/bbs/Gossiping/index.html'
car = 'https://www.ptt.cc/bbs/car/index.html'
creditcard = 'https://www.ptt.cc/bbs/creditcard/index.html'

In [33]:
#wb = xw.Book('ptt_mobile.xlsx') #讀取檔案
wb = xw.Book()
wb.save('output.xlsx')

In [34]:
sheet = wb.sheets[0]

In [35]:
sheet.range('A1').value = [['發布日期', '發布時間', '來源', '主文tag', '網址', '主文作者', '標題', '主文','回文tag', '回文作者', '回文', '回文時間', '主文/回文']]

In [36]:
# 選取愈抓取日期, (start date, end date), end date 需為當日日期, 格式 03/1,03/10
today = date.today().strftime("%m/%d")
days = getEveryDay('04/12',today)
days

['04/12', '04/13']

In [37]:
# 依據date選取爬取頁面 (days, current_page)
page = get_page_url(days, car)
page

['https://www.ptt.cc/bbs/car/index.html',
 'https://www.ptt.cc/bbs/car/index5009.html',
 'https://www.ptt.cc/bbs/car/index5008.html',
 'https://www.ptt.cc/bbs/car/index5007.html',
 'https://www.ptt.cc/bbs/car/index5006.html']

In [38]:
current_row = 2
for p in page:
    articles = ppt_web_crawler(p,days)
    saved_content = cp_content(articles)
    sheet.range('A' + str(current_row)).value = saved_content
    current_row += len(saved_content)
print('\n ***** Finish *****')
    

[新聞] Toyota Crown全車系到齊！三款規格揭露
[問題] BMW F06 640i 改裝推薦
[問題] 抬頭顯示器+胎壓偵測整合推薦？
[新聞] 新世代皮卡Ford Ranger開放預接單
Re: [新聞] 新世代皮卡Ford Ranger開放預接單
[心得] Prius 5 心得(日本自駕)
[討論] 4/18上海車展-保時捷
[討論] Prius PHEV怎麼不出休旅車
[問車] 小休旅選擇請益
[分享] 下午差點遲到的親身經歷
[討論] 開名貴的車會閃比較遠?
[問題] 台灣賓士能主導核章？
Re: [分享] Ford 6.5代野馬一年油耗和保養分享
[問題] 請問Toyota Camry 2008警報
[問題] ALTIS 12代 排檔異音?
[問題] 為何最近比較少看到竊車新聞?
[心得] Yaris 經典版 購車心得
[討論] 現在買2020的model 3會很盤嗎?
[討論] T-Roc 330改裝 or 直上R
Re: [討論] T-Roc 330改裝 or 直上R
[菜單] 2023 Corolla Cross+Corolla Altis
Re: [分享] Prius5可以訂車了
[情報] GM正逐步取消AA跟CarPlay 改原生Android
[問題] 稅制保護國產是不是要重新檢視國產成分？
Re: [分享] Prius5可以訂車了
[問題] 請問哪家超額險能保三千萬以上?
[新聞] Lexus UX 300e大電池改款版本國內上市
[分享] 自動駕駛計程車搭乘意願調查（抽商店禮卷
[心得] 現代汽車 Tucson L b款 我當冤大頭的經過
[新聞] MG Cyberster電動小跑車 500hp動力有點搞
[新聞] 探頭確認過！擎天崗遊客「超認真倒車」
[問題]領牌車的付款流程請益
Re: [閒聊] 新竹某國小前面
[問題] 插隊問題
[問題] 隔熱紙請益
[新聞] Range Rover SV正式在台上市，售價845萬
[問題] RX200t 跑的贏 330i 嗎?
[電車] Model 3 HIGHLAND 流出
[討論] 國道休息站停車場應該要收費嗎？
Re: [閒聊] 新竹某國小前面
Re: [問題] 為什麼特斯拉造肇事率這麼高?
[新聞] 行人地獄再現！駕駛無視導護闖紅燈 差點撞上斑馬線學童
[新聞] Toyota最新

In [19]:
wb.save()

In [20]:
wb.close()

## Test